# Projeto Programação para a Ciência de Dados
Urgências e Hospitais em Portugal

## Importar as bibliotecas necessárias

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn import metrics
 
# mostrar gráficos no notebook
%matplotlib inline

## Exportar os datesets necessários para o projeto
Os nossos datasets foram descarregados através do Portal da Transparência do Serviço Nacional de Saúde (https://transparencia.sns.gov.pt/pages/home-page/) e estão localizados na pasta "datasets".
Começamos por ler os ficheiros csv e transformá-los em dataframes através da biblioteca Pandas para podermos manipular dados.

In [33]:
urgencias = pd.read_csv('datasets/atendimentos-por-urgencia.csv', sep=';', encoding="utf-8")
morbilidade = pd.read_csv('datasets/morbilidade.csv', sep=';', encoding="utf-8")

print("Dados de Urgências:")
urgencias.info()

print("\nDados de Morbilidade:")
morbilidade.info()

Dados de Urgências:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5841 entries, 0 to 5840
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Período                 5841 non-null   object 
 1   Período.Format.2        5841 non-null   object 
 2   Região                  5841 non-null   object 
 3   Instituição             5841 non-null   object 
 4   Localização Geográfica  5841 non-null   object 
 5   Urgências Geral         5840 non-null   float64
 6   Urgências Pediátricas   5111 non-null   float64
 7   Urgência Obstetricia    4964 non-null   float64
 8   Urgência Psiquiátrica   440 non-null    float64
 9   Total Urgências         5840 non-null   float64
dtypes: float64(5), object(5)
memory usage: 456.5+ KB

Dados de Morbilidade:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480554 entries, 0 to 480553
Data columns (total 11 columns):
 #   Column                                    No

In [34]:
urgencias.head()

,Período,Período.Format.2,Região,Instituição,Localização Geográfica,Urgências Geral,Urgências Pediátricas,Urgência Obstetricia,Urgência Psiquiátrica,Total Urgências
0,2024-01,2024/01/01,Região de Saúde LVT,"Unidade Local de Saúde da Arrábida, EPE","38.529351, -8.881073",5433.0,2425.0,515.0,NaN,8373.0
1,2024-01,2024/01/01,Região de Saúde Norte,Unidade Local de Saúde da Póvoa de Varzim/Vila...,"41.383183, -8.758784",5536.0,NaN,NaN,NaN,5536.0
2,2024-01,2024/01/01,Região de Saúde LVT,"Unidade Local de Saúde de Amadora/Sintra, EPE","38.7436332060551, -9.0",13917.0,3268.0,1303.0,NaN,18488.0
3,2024-01,2024/01/01,Região de Saúde Norte,"Unidade Local de Saúde de Braga, EPE","41.562328338623, -8.3",11752.0,4087.0,998.0,NaN,16837.0
4,2024-01,2024/01/01,Região de Saúde Norte,"Unidade Local de Saúde de Entre Douro e Vouga,...","40.9295, -8.54567",15899.0,2720.0,630.0,NaN,19249.0


In [35]:
morbilidade.head()

,Período,Código Capítulo Diagnóstico Principal,Instituição,Regiões,Descrição Capítulo Diagnóstico Principal,Faixa Etária,Sexo,Internamentos,Dias de Internamento,Ambulatório,Óbitos
0,2019-07,1,"Unidade Local de Saúde do Oeste, E.P.E.",Região de Saúde LVT,Algumas doenças infecciosas e parasitárias,[5-15[,F,1,1,0,0
1,2019-07,1,"Unidade Local de Saúde do Tâmega e Sousa, E.P.E.",Região de Saúde do Norte,Algumas doenças infecciosas e parasitárias,[1-5[,M,3,14,0,0
2,2019-07,1,"Unidade Local de Saúde do Tâmega e Sousa, E.P.E.",Região de Saúde do Norte,Algumas doenças infecciosas e parasitárias,[45-65[,F,2,12,0,1
3,2019-07,1,"Unidade Local de Saúde do Tâmega e Sousa, E.P.E.",Região de Saúde do Norte,Algumas doenças infecciosas e parasitárias,[5-15[,F,2,7,0,0
4,2019-07,2,Hospital de Cascais Dr. José de Almeida,Região de Saúde LVT,Neoplasias,[45-65[,F,8,264,2,1


## Limpeza dos Dados
### Remover colunas desnecessárias
Começamos a etapa de modelação dos dados com a remoção de colunas que serão desnecessárias para o nosso projeto, a coluna "Período.Format.2" na dataframe das urgências apenas mostra a data num formato diferente que não é necessário para o que queremos no nosso modelo, por isso vamos removê-la.

In [36]:
urgencias = urgencias.drop(columns=['Período.Format.2'])
urgencias = urgencias.drop(columns=['Localização Geográfica'])

urgencias.head()

,Período,Região,Instituição,Urgências Geral,Urgências Pediátricas,Urgência Obstetricia,Urgência Psiquiátrica,Total Urgências
0,2024-01,Região de Saúde LVT,"Unidade Local de Saúde da Arrábida, EPE",5433.0,2425.0,515.0,NaN,8373.0
1,2024-01,Região de Saúde Norte,Unidade Local de Saúde da Póvoa de Varzim/Vila...,5536.0,NaN,NaN,NaN,5536.0
2,2024-01,Região de Saúde LVT,"Unidade Local de Saúde de Amadora/Sintra, EPE",13917.0,3268.0,1303.0,NaN,18488.0
3,2024-01,Região de Saúde Norte,"Unidade Local de Saúde de Braga, EPE",11752.0,4087.0,998.0,NaN,16837.0
4,2024-01,Região de Saúde Norte,"Unidade Local de Saúde de Entre Douro e Vouga,...",15899.0,2720.0,630.0,NaN,19249.0


### Renomear colunas
De seguida, mudamos o nome das colunas para estarem todas sem acentos e em letra pequena.

In [37]:
morbilidade = morbilidade.rename(columns={
    'Período': 'periodo',
    'Regiões': 'regiao',
    'Código Capítulo Diagnóstico Principal': 'codigo_diagnostico',
    'Descrição Capítulo Diagnóstico Principal': 'diagnostico',
    'Faixa Etária': 'faixa_etaria',
    'Sexo': 'sexo',
    'Internamentos': 'internamentos',
    'Instituição': 'instituicao',
    'Dias de Internamento': 'dias_internamento',
    'Ambulatório': 'ambulatorio',
    'Óbitos': 'obitos',
})

urgencias = urgencias.rename(columns={
    'Período': 'periodo',
    'Região': 'regiao',
    'Instituição': 'instituicao',
    'Urgências Geral': 'urgencias_geral',
    'Urgências Pediátricas': 'urgencias_pediatricas',
    'Urgência Obstétrica': 'urgencias_obstetricas',
    'Urgência Psiquiátrica': 'urgencias_psiquiatricas',
    'Total Urgências': 'total_urgencias'
})

### Trocar tipos de dados
Não é possivel trocar o tipo de dados das urgencias para int porque temos valores em falta, e como não sabemos se estes valores estão em falta porque as instituições não aceitam um tipo de urgência ou porque realmente não houve aquele tipo de urgências, por isso não tocamos nestes valores.

In [38]:
urgencias['periodo'] = pd.to_datetime(urgencias['periodo'])
morbilidade['periodo'] = pd.to_datetime(morbilidade['periodo'])

print("Tipos de dados em urgências:")
print(urgencias.dtypes)
print("\nTipos de dados em morbilidade:")
print(morbilidade.dtypes)

Tipos de dados em urgências:
periodo                    datetime64[ns]
regiao                             object
instituicao                        object
urgencias_geral                   float64
urgencias_pediatricas             float64
Urgência Obstetricia              float64
urgencias_psiquiatricas           float64
total_urgencias                   float64
dtype: object

Tipos de dados em morbilidade:
periodo               datetime64[ns]
codigo_diagnostico             int64
instituicao                   object
regiao                        object
diagnostico                   object
faixa_etaria                  object
sexo                          object
internamentos                  int64
dias_internamento              int64
ambulatorio                    int64
obitos                         int64
dtype: object


### Remover letras e caracteres desnecessárias
Podemos reparar que a coluna da faixa etária tem parentesis retos desnecessários, então começamos por os remover. De seguida, iremos também remover a indicação do modelo de gestão de cada hospital pois não faz sentido para o nosso estudo e irá permitir uma melhor padronização dos dados mais tarde.

In [44]:
morbilidade['faixa_etaria'] = morbilidade['faixa_etaria'].str.replace('[', '')
morbilidade['faixa_etaria'] = morbilidade['faixa_etaria'].str.replace(']', '')

morbilidade['instituicao'] = morbilidade['instituicao'].str.split(',').str[0]
urgencias['instituicao'] = urgencias['instituicao'].str.split(',').str[0]

morbilidade['regiao'] = morbilidade['regiao'].str.replace(' do ', ' ', regex=False)
urgencias['regiao'] = urgencias['regiao'].str.replace(' do ', ' ', regex=False)

morbilidade

,periodo,codigo_diagnostico,instituicao,regiao,diagnostico,faixa_etaria,sexo,internamentos,dias_internamento,ambulatorio,obitos
0,2019-07-01,1,Unidade Local de Saúde do Oeste,Região de Saúde LVT,Algumas doenças infecciosas e parasitárias,5-15,F,1,1,0,0
1,2019-07-01,1,Unidade Local de Saúde do Tâmega e Sousa,Região de Saúde Norte,Algumas doenças infecciosas e parasitárias,1-5,M,3,14,0,0
2,2019-07-01,1,Unidade Local de Saúde do Tâmega e Sousa,Região de Saúde Norte,Algumas doenças infecciosas e parasitárias,45-65,F,2,12,0,1
3,2019-07-01,1,Unidade Local de Saúde do Tâmega e Sousa,Região de Saúde Norte,Algumas doenças infecciosas e parasitárias,5-15,F,2,7,0,0
4,2019-07-01,2,Hospital de Cascais Dr. José de Almeida,Região de Saúde LVT,Neoplasias,45-65,F,8,264,2,1
...,...,...,...,...,...,...,...,...,...,...,...
480549,2019-12-01,7,Unidade Local de Saúde do Alto Minho,Região de Saúde Norte,Doenças do olho e anexos,25-45,M,0,0,1,0
480550,2019-12-01,7,Unidade Local de Saúde do Baixo Alentejo,Região de Saúde Alentejo,Doenças do olho e anexos,25-45,M,1,1,0,0
480551,2019-12-01,7,Unidade Local de Saúde do Baixo Alentejo,Região de Saúde Alentejo,Doenças do olho e anexos,65-120,M,0,0,6,0
480552,2019-12-01,7,Unidade Local de Saúde do Baixo Mondego,Região de Saúde Centro,Doenças do olho e anexos,25-45,F,0,0,1,0


### Remover todos os dados antes de 2019
O dataset da morbilidade apenas tem dados para os anos a partir de 2019 então vamos apagar todos os dados até 2019 do dataset das urgencias

In [ ]:
urgencias = urgencias[urgencias['periodo'] >= '2019-01-01']

urgencias.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2960 entries, 0 to 5840
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   periodo                  2960 non-null   datetime64[ns]
 1   regiao                   2960 non-null   object        
 2   instituicao              2960 non-null   object        
 3   urgencias_geral          2959 non-null   float64       
 4   urgencias_pediatricas    2590 non-null   float64       
 5   Urgência Obstetricia     2516 non-null   float64       
 6   urgencias_psiquiatricas  294 non-null    float64       
 7   total_urgencias          2959 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 272.7+ KB


### Remover instituições que não têm morbilidade e vice-versa
Sendo estes dois datasets diferentes, pode haver instancias onde uma instituição pode não ter registo de urgências ou não ter registo de óbitos, o que faz com que estas instituições sejam irelevantes para o nosso estudo, com isto, será necessário remover estes dados desnecessários

In [59]:
instituicoes_urgencias = urgencias['instituicao'].unique()
instituicoes_morbilidade = morbilidade['instituicao'].unique()

instituicoes_comum = set(instituicoes_urgencias) & set(instituicoes_morbilidade)

urgencias = urgencias[urgencias['instituicao'].isin(instituicoes_comum)]
morbilidade = morbilidade[morbilidade['instituicao'].isin(instituicoes_comum)]


print("Instituições comuns em urgências e morbilidade:")
print(urgencias['instituicao'].unique())

print("\nInstituições comuns em morbilidade:")
print(morbilidade['instituicao'].unique())

morbilidade


Instituições comuns em urgências e morbilidade:
['Unidade Local de Saúde da Arrábida' 'Unidade Local de Saúde de Braga'
 'Unidade Local de Saúde de Entre Douro e Vouga'
 'Unidade Local de Saúde de Matosinhos'
 'Unidade Local de Saúde do Médio Tejo'
 'Unidade Local de Saúde da Lezíria'
 'Unidade Local de Saúde da Região de Leiria'
 'Unidade Local de Saúde do Alto Minho'
 'Unidade Local de Saúde do Estuário do Tejo'
 'Unidade Local de Saúde da Região de Aveiro'
 'Unidade Local de Saúde de Santa Maria'
 'Unidade Local de Saúde de Santo António'
 'Unidade Local de Saúde de São José' 'Unidade Local de Saúde do Alto Ave'
 'Unidade Local de Saúde do Baixo Mondego'
 'Unidade Local de Saúde do Litoral Alentejano'
 'Unidade Local de Saúde do Médio Ave' 'Unidade Local de Saúde da Guarda'
 'Unidade Local de Saúde do Baixo Alentejo'
 'Unidade Local de Saúde da Cova da Beira'
 'Unidade Local de Saúde de Lisboa Ocidental'
 'Unidade Local de Saúde de São João'
 'Unidade Local de Saúde de Trás-os-Monte

,periodo,codigo_diagnostico,instituicao,regiao,diagnostico,faixa_etaria,sexo,internamentos,dias_internamento,ambulatorio,obitos
0,2019-07-01,1,Unidade Local de Saúde do Oeste,Região de Saúde LVT,Algumas doenças infecciosas e parasitárias,5-15,F,1,1,0,0
1,2019-07-01,1,Unidade Local de Saúde do Tâmega e Sousa,Região de Saúde Norte,Algumas doenças infecciosas e parasitárias,1-5,M,3,14,0,0
2,2019-07-01,1,Unidade Local de Saúde do Tâmega e Sousa,Região de Saúde Norte,Algumas doenças infecciosas e parasitárias,45-65,F,2,12,0,1
3,2019-07-01,1,Unidade Local de Saúde do Tâmega e Sousa,Região de Saúde Norte,Algumas doenças infecciosas e parasitárias,5-15,F,2,7,0,0
7,2019-07-01,4,Unidade Local de Saúde de Coimbra,Região de Saúde Centro,"Doenças endócrinas, nutricionais e metabólicas",15-25,M,3,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...
480549,2019-12-01,7,Unidade Local de Saúde do Alto Minho,Região de Saúde Norte,Doenças do olho e anexos,25-45,M,0,0,1,0
480550,2019-12-01,7,Unidade Local de Saúde do Baixo Alentejo,Região de Saúde Alentejo,Doenças do olho e anexos,25-45,M,1,1,0,0
480551,2019-12-01,7,Unidade Local de Saúde do Baixo Alentejo,Região de Saúde Alentejo,Doenças do olho e anexos,65-120,M,0,0,6,0
480552,2019-12-01,7,Unidade Local de Saúde do Baixo Mondego,Região de Saúde Centro,Doenças do olho e anexos,25-45,F,0,0,1,0


### Padronizar campos
O atributo da instituição e região em ambos os datasets são diferentes, ás vezes tem um "de" ou um "do" a mais (Região de Saúde do Norte e Região de Saúde Norte), ou tem pontos a mais a separar as letras do tipo de modelo de gestão da instituição (EPE e E.P.E.), por isso vamos começar por padronizar estes campos para deixar de ter "de", "do" e "da" na região, e remover todos os pontos no tipo de modelo de gestão.